# 0
### A)

In [15]:
from pathlib import Path
from dotenv import load_dotenv
import requests
import os

In [16]:
env_path = Path("../../code_alongs/11_data_from_API/.env")
load_dotenv(env_path)


API_KEY = os.getenv("API_KEY")

url = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000002&destId=740000003&passlist=true&showPassingPoints=true&accessId={API_KEY}"

response = requests.get(url)

result = response.json()



result.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [17]:
def get_departures(api):
    url = f"https://api.resrobot.se/v2.1/departureBoard?id=740000002&format=json&accessId={api}"
    response = requests.get(url)
    return response.json()


def get_arrivals(api):
    url = f"https://api.resrobot.se/v2.1/arrivalBoard?id=740000002&format=json&accessId={api}"
    response = requests.get(url)
    return response.json()

In [18]:
departures = get_departures(api=API_KEY)
arrivals = get_arrivals(api=API_KEY)

In [19]:
departures.keys(), arrivals.keys()

(dict_keys(['Departure', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId']),
 dict_keys(['Arrival', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId']))

### B)

In [20]:
len(arrivals["Arrival"])

170

### C)

In [21]:
len(departures["Departure"])

176

### D)

In [22]:
departures["Departure"][0].keys()

dict_keys(['JourneyDetailRef', 'JourneyStatus', 'ProductAtStop', 'Product', 'Notes', 'name', 'type', 'stop', 'stopid', 'stopExtId', 'lon', 'lat', 'time', 'date', 'reachable', 'direction', 'directionFlag'])

In [23]:
for depart in departures["Departure"]:
    if "Spårväg" in depart["name"]:
        print(f"{depart['name']}, {depart['direction']}, {depart['time']}")

Länstrafik - Spårväg 1, Tynnered Opaltorget (Göteborg kn), 13:30:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 13:36:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 13:39:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 13:46:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 13:49:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 13:55:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 13:59:00
Länstrafik - Spårväg 2, Mölndals Innerstad, 14:00:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 14:03:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 14:10:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 14:11:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 14:19:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 14:20:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 14:28:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 14:30:00


### E)

In [24]:
# point map över spårvagnar

### F)

# 1
### A)

# Stop Lookup

In [42]:
def get_lookup(search):
    url= f"https://api.resrobot.se/v2.1/location.name?input={search}&format=json&accessId={API_KEY}"
    return requests.get(url).json()

In [58]:
lund= get_lookup("Lund")
lund["stopLocationOrCoordLocation"][0]["StopLocation"]["extId"]

'740098038'

In [61]:
gothenburg= get_lookup("Göteborg")
gothenburg["stopLocationOrCoordLocation"][0]["StopLocation"]["extId"]

'740098001'

In [59]:
def route_planner(org, des):
    
    url= f"https://api.resrobot.se/v2.1/trip?format=json&originId={org}&destId={des}&passlist=true&showPassingPoints=true&accessId={API_KEY}"

    return requests.get(url).json()

In [62]:
trip= route_planner(740098001, 740098038)

In [66]:
trip.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [73]:
example_trip = trip["Trip"][0]

In [85]:
stops=example_trip["LegList"]["Leg"][0]["Stops"]["Stop"]
stops

[{'name': 'Göteborg Centralstation',
  'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
  'extId': '740000002',
  'routeIdx': 0,
  'lon': 11.973479,
  'lat': 57.708895,
  'depPrognosisType': 'PROGNOSED',
  'depTime': '14:40:00',
  'depDate': '2025-01-09',
  'depDir': 'Köpenhamn Österport (Danmark)',
  'minimumChangeDuration': 'PT15M'},
 {'name': 'Mölndal station',
  'id': 'A=1@O=Mölndal station@X=12018722@Y=57655958@U=1@L=740000315@',
  'extId': '740000315',
  'routeIdx': 1,
  'lon': 12.018722,
  'lat': 57.655958,
  'depTime': '14:48:00',
  'depDate': '2025-01-09',
  'arrTime': '14:48:00',
  'arrDate': '2025-01-09',
  'minimumChangeDuration': 'PT9M'},
 {'name': 'Kungsbacka station',
  'id': 'A=1@O=Kungsbacka station@X=12079614@Y=57490556@U=1@L=740000161@',
  'extId': '740000161',
  'routeIdx': 2,
  'lon': 12.079614,
  'lat': 57.490556,
  'depTime': '14:59:00',
  'depDate': '2025-01-09',
  'arrTime': '14:59:00',
  'arrDate': '2025-01-09',
  'minimumChangeDur

In [86]:
[stop.get("name")for stop in stops]

['Göteborg Centralstation',
 'Mölndal station',
 'Kungsbacka station',
 'Varberg station',
 'Falkenberg station',
 'Halmstad Centralstation',
 'Laholm station',
 'Båstad station',
 'Ängelholm station',
 'Helsingborg Centralstation',
 'Landskrona station',
 'Lund Centralstation']

### B)